<a href="https://colab.research.google.com/github/NassimZahri/Data_Mining/blob/main/01_importer_nettoyer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 — Importer & Nettoyer les Données
Dans ce notebook, vous apprendrez à charger des données (CSV/Excel/JSON), inspecter les types, gérer les valeurs manquantes/duplicatas, parser des dates et optimiser la mémoire.

**Données utilisées :** `data/ventes.csv`, `data/clients.csv`, `data/produits.csv`

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

#DATA_DIR = Path('data')
ventes = pd.read_csv('data/ventes.csv', parse_dates=['date'])
clients = pd.read_csv('data/clients.csv')
produits = pd.read_csv('data/produits.csv')

ventes.head()


,date,store,city,product_id,price,quantity,promo,total
0,2023-01-01,Magasin_17,Marrakech,89,140.93,4,0,563.72
1,2023-01-01,Magasin_08,Fès,88,32.44,2,1,64.88
2,2023-01-01,Magasin_20,Fès,42,35.78,4,0,143.12
3,2023-01-01,Magasin_19,Rabat,40,157.74,2,0,315.48
4,2023-01-01,Magasin_15,Agadir,45,175.49,4,0,701.96


## 1. Aperçu et informations

In [ ]:
ventes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        2000 non-null   datetime64[ns]
 1   store       2000 non-null   object        
 2   city        1951 non-null   object        
 3   product_id  2000 non-null   int64         
 4   price       1940 non-null   float64       
 5   quantity    2000 non-null   int64         
 6   promo       2000 non-null   int64         
 7   total       1940 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 125.1+ KB


In [ ]:
clients.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       800 non-null    int64  
 1   age             800 non-null    int64  
 2   city            800 non-null    object 
 3   revenu_mensuel  772 non-null    float64
 4   churn           800 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 31.4+ KB


## 2. Statistiques descriptives

In [ ]:
ventes.describe(include='all')


,date,store,city,product_id,price,quantity,promo,total
count,2000,2000,1951,2000.000000,1940.000000,2000.000000,2000.000000,1940.000000
unique,NaN,20,8,NaN,NaN,NaN,NaN,NaN
top,NaN,Magasin_17,Meknès,NaN,NaN,NaN,NaN,NaN
freq,NaN,122,266,NaN,NaN,NaN,NaN,NaN
mean,2023-10-20 07:02:38.399999744,NaN,NaN,51.267000,128.857526,3.644000,0.228500,475.542402
min,2023-01-01 00:00:00,NaN,NaN,1.000000,5.150000,0.000000,0.000000,0.000000
25%,2023-05-24 00:00:00,NaN,NaN,26.000000,69.447500,2.000000,0.000000,134.295000
50%,2023-10-18 12:00:00,NaN,NaN,52.000000,130.465000,3.000000,0.000000,309.370000
75%,2024-03-17 00:00:00,NaN,NaN,77.000000,189.320000,4.000000,0.000000,577.370000
max,2024-08-21 00:00:00,NaN,NaN,100.000000,249.940000,150.000000,1.000000,23980.500000


## 3. Détection des valeurs manquantes

In [ ]:
ventes.isna().mean().sort_values(ascending=False)


,0
total,0.0300
price,0.0300
city,0.0245
date,0.0000
product_id,0.0000
store,0.0000
quantity,0.0000
promo,0.0000


### 3.1 Stratégies de traitement

In [ ]:
# Exemple : imputation simple
ventes_clean = ventes.copy()
# Impute le prix par la médiane par produit
med_by_prod = ventes_clean.groupby('product_id')['price'].transform(lambda s: s.fillna(s.median()))
ventes_clean['price'] = np.where(ventes_clean['price'].isna(), med_by_prod, ventes_clean['price'])

# Impute la ville manquante par 'Inconnue'
ventes_clean['city'] = ventes_clean['city'].fillna('Inconnue')
ventes_clean.head()


,date,store,city,product_id,price,quantity,promo,total
0,2023-01-01,Magasin_17,Marrakech,89,140.93,4,0,563.72
1,2023-01-01,Magasin_08,Fès,88,32.44,2,1,64.88
2,2023-01-01,Magasin_20,Fès,42,35.78,4,0,143.12
3,2023-01-01,Magasin_19,Rabat,40,157.74,2,0,315.48
4,2023-01-01,Magasin_15,Agadir,45,175.49,4,0,701.96


## 4. Gestion des doublons

In [ ]:
nb_avant = len(ventes_clean)
ventes_clean = ventes_clean.drop_duplicates(subset=['date','store','product_id'], keep='first')
nb_apres = len(ventes_clean)
print('Doublons supprimés :', nb_avant - nb_apres)


Doublons supprimés : 0


## 5. Parsing de dates & création de features temporelles

In [ ]:
ventes_clean['year'] = ventes_clean['date'].dt.year
ventes_clean['month'] = ventes_clean['date'].dt.month
ventes_clean['dow'] = ventes_clean['date'].dt.dayofweek
ventes_clean[['date','year','month','dow']].head()


,date,year,month,dow
0,2023-01-01,2023,1,6
1,2023-01-01,2023,1,6
2,2023-01-01,2023,1,6
3,2023-01-01,2023,1,6
4,2023-01-01,2023,1,6


## 6. Optimisation mémoire (downcasting)

In [ ]:
def downcast_df(df):
    for c in df.select_dtypes(include=['int64','int32']).columns:
        df[c] = pd.to_numeric(df[c], downcast='integer')
    for c in df.select_dtypes(include=['float64','float32']).columns:
        df[c] = pd.to_numeric(df[c], downcast='float')
    return df

print('Mémoire avant:', ventes_clean.memory_usage(deep=True).sum()/1e6, 'MB')
ventes_opt = downcast_df(ventes_clean.copy())
print('Mémoire après:', ventes_opt.memory_usage(deep=True).sum()/1e6, 'MB')


Mémoire avant: 0.366715 MB
Mémoire après: 0.294715 MB


## 7. EXERCICE
- Supprimez les lignes où `quantity` est négative puis remplacez les valeurs extrêmes (au-delà du 99e percentile) par la valeur du 99e percentile.
- Créez une fonction `clean_prices(df)` qui : (1) impute `price` par produit, (2) filtre `price<=0` et (3) ajoute une colonne `log_price`.
*(Ajoutez vos cellules ci-dessous)*

In [ ]:
# Supprimez les lignes où quantity est négative
ventes_filtered = ventes_opt[ventes_opt['quantity'] >= 0].copy()

# Remplacez les valeurs extrêmes (au-delà du 99e percentile) par la valeur du 99e percentile.
percentile_99 = ventes_filtered['quantity'].quantile(0.99)
ventes_filtered['quantity'] = np.where(ventes_filtered['quantity'] > percentile_99, percentile_99, ventes_filtered['quantity'])

print("DataFrame après filtrage et remplacement des valeurs extrêmes :")
display(ventes_filtered.describe())

DataFrame après filtrage et remplacement des valeurs extrêmes :


,date,product_id,price,quantity,promo,total,year,month,dow
count,2000,2000.000000,2000.000000,2000.000000,2000.000000,1940.000000,2000.000000,2000.00000,2000.000000
mean,2023-10-20 07:02:38.399999744,51.267000,129.167160,3.058000,0.228500,475.542389,2023.374000,5.66250,3.100000
min,2023-01-01 00:00:00,1.000000,5.150000,0.000000,0.000000,0.000000,2023.000000,1.00000,0.000000
25%,2023-05-24 00:00:00,26.000000,70.764999,2.000000,0.000000,134.294998,2023.000000,3.00000,1.000000
50%,2023-10-18 12:00:00,52.000000,131.990005,3.000000,0.000000,309.369995,2023.000000,5.00000,3.000000
75%,2024-03-17 00:00:00,77.000000,187.605000,4.000000,0.000000,577.369995,2024.000000,8.00000,5.000000
max,2024-08-21 00:00:00,100.000000,249.940002,9.000000,1.000000,23980.500000,2024.000000,12.00000,6.000000
std,NaN,28.925042,70.130875,1.778544,0.419971,1239.361572,0.483985,3.24898,2.020654


# Explication étape par étape du Notebook

Ce notebook est dédié aux premières étapes de nettoyage et de préparation de données de ventes, clients et produits.

## Étape 1: Importer & Nettoyer les Données

Cette section commence par l'importation des bibliothèques nécessaires comme `pandas` pour la manipulation de données et `numpy` pour les opérations numériques. Elle charge ensuite trois fichiers CSV (`ventes.csv`, `clients.csv`, `produits.csv`) dans des DataFrames pandas. Pour le DataFrame `ventes`, la colonne 'date' est directement parsée en format de date lors du chargement. Les premières lignes du DataFrame `ventes` sont ensuite affichées pour un aperçu.

## Étape 2: Aperçu et informations

Ici, la méthode `.info()` est utilisée sur les DataFrames `ventes` et `clients`. Cela permet d'obtenir un résumé concis de chaque DataFrame, y compris le nombre d'entrées, le nom des colonnes, le nombre de valeurs non nulles dans chaque colonne, le type de données de chaque colonne et l'utilisation de la mémoire. C'est une étape cruciale pour identifier les colonnes avec des valeurs manquantes et comprendre les types de données.

## Étape 3: Statistiques descriptives

La méthode `.describe(include='all')` est appliquée au DataFrame `ventes`. Cette méthode génère des statistiques descriptives. Pour les colonnes numériques, elle affiche le compte, la moyenne, l'écart-type, les valeurs minimum et maximum, et les quartiles. Pour les colonnes non numériques (comme les objets), elle affiche le compte, le nombre d'éléments uniques, la valeur la plus fréquente (`top`) et sa fréquence (`freq`).

## Étape 4: Détection des valeurs manquantes

Cette étape calcule le pourcentage de valeurs manquantes pour chaque colonne du DataFrame `ventes`. En utilisant `ventes.isna().mean()`, on obtient la proportion de valeurs `NaN` dans chaque colonne, triée par ordre décroissant pour identifier rapidement les colonnes avec le plus de valeurs manquantes ('total', 'price', 'city').

## Étape 5: Stratégies de traitement des valeurs manquantes

Un exemple de gestion des valeurs manquantes est présenté. Pour la colonne 'price', les valeurs manquantes sont imputées par la médiane du prix pour chaque 'product_id'. Pour la colonne 'city', les valeurs manquantes sont simplement remplacées par la chaîne 'Inconnue'. Le début du DataFrame `ventes_clean` (une copie du DataFrame original après imputation) est affiché.

## Étape 6: Gestion des doublons

Cette section vérifie la présence et supprime les lignes en double dans le DataFrame `ventes_clean` en se basant sur la combinaison des colonnes 'date', 'store', et 'product_id'. Le nombre de doublons supprimés est affiché. Dans ce cas, il n'y en a pas.

## Étape 7: Parsing de dates & création de features temporelles

Les composantes temporelles (année, mois et jour de la semaine) sont extraites de la colonne 'date' du DataFrame `ventes_clean` et ajoutées comme de nouvelles colonnes nommées 'year', 'month', et 'dow'. Les premières lignes avec ces nouvelles colonnes sont affichées.

## Étape 8: Optimisation mémoire (downcasting)

Une fonction `downcast_df` est définie pour réduire l'utilisation de la mémoire du DataFrame. Elle itère sur les colonnes numériques (entiers et flottants) et tente de les convertir en types de données plus petits si possible (par exemple, `int64` à `int32`). L'utilisation de la mémoire avant et après l'application de cette fonction est affichée pour montrer l'efficacité de l'optimisation.

## Étape 9: EXERCICE

Cette section présente un exercice pratique en deux parties :

1.  **Nettoyage de la quantité**: Les lignes où la colonne 'quantity' a une valeur négative sont supprimées. Ensuite, les valeurs extrêmes dans la colonne 'quantity' (celles au-delà du 99ème percentile) sont remplacées par la valeur du 99ème percentile. Les statistiques descriptives du DataFrame résultant sont affichées.
2.  **Fonction de nettoyage des prix**: Une fonction nommée `clean_prices` est définie. Cette fonction prend un DataFrame en entrée, impute les valeurs manquantes dans la colonne 'price' par la médiane du prix par 'product_id', filtre les lignes où le prix est inférieur ou égal à zéro, et ajoute une nouvelle colonne 'log_price' qui est le logarithme du prix. La fonction est ensuite appliquée et les statistiques descriptives du DataFrame après nettoyage des prix sont affichées.

In [ ]:
def clean_prices(df):
    """
    Clean the 'price' column in a DataFrame.

    Args:
        df: pandas DataFrame with a 'price' and 'product_id' column.

    Returns:
        pandas DataFrame with cleaned 'price' and a new 'log_price' column.
    """
    df_clean = df.copy()

    # (1) Impute price by product
    med_by_prod = df_clean.groupby('product_id')['price'].transform(lambda s: s.fillna(s.median()))
    df_clean['price'] = np.where(df_clean['price'].isna(), med_by_prod, df_clean['price'])

    # (2) Filter price <= 0
    df_clean = df_clean[df_clean['price'] > 0].copy()

    # (3) Add a log_price column
    df_clean['log_price'] = np.log(df_clean['price'])

    return df_clean

# Apply the function to the filtered dataframe
ventes_cleaned_prices = clean_prices(ventes_filtered)

print("\nDataFrame après nettoyage des prix :")
display(ventes_cleaned_prices.describe())


DataFrame après nettoyage des prix :


,date,product_id,price,quantity,promo,total,year,month,dow,log_price
count,2000,2000.000000,2000.000000,2000.000000,2000.000000,1940.000000,2000.000000,2000.00000,2000.000000,2000.000000
mean,2023-10-20 07:02:38.399999744,51.267000,129.167160,3.058000,0.228500,475.542389,2023.374000,5.66250,3.100000,4.614920
min,2023-01-01 00:00:00,1.000000,5.150000,0.000000,0.000000,0.000000,2023.000000,1.00000,0.000000,1.638997
25%,2023-05-24 00:00:00,26.000000,70.764999,2.000000,0.000000,134.294998,2023.000000,3.00000,1.000000,4.259364
50%,2023-10-18 12:00:00,52.000000,131.990005,3.000000,0.000000,309.369995,2023.000000,5.00000,3.000000,4.882726
75%,2024-03-17 00:00:00,77.000000,187.605000,4.000000,0.000000,577.369995,2024.000000,8.00000,5.000000,5.234339
max,2024-08-21 00:00:00,100.000000,249.940002,9.000000,1.000000,23980.500000,2024.000000,12.00000,6.000000,5.521221
std,NaN,28.925042,70.130875,1.778544,0.419971,1239.361572,0.483985,3.24898,2.020654,0.834385
